In [21]:
#!pip install tweet-preprocessor

  Running setup.py bdist_wheel for tweet-preprocessor ... done
  Stored in directory: /home/jovyan/.cache/pip/wheels/64/d9/5c/c72815d52f5770f229b909840516fd7a36e11bd57d4b8f28dd
Successfully built tweet-preprocessor


In [ ]:
pd.options.display.max_colwidth = 75

In [3]:
pwd

/home/jovyan


In [16]:
cd eg_twitter_raw/


/home/jovyan/capstone-34/34.210.104.115/eg_twitter_raw


In [4]:
import preprocessor as p

In [24]:
import pandas as pd
import json
import re

### Turn query documents (tweets) into generators

In [17]:
def load_tweets(file):
    with open(file, 'r') as f:
        tweets = (json.loads(line) for line in f.readlines())
    return tweets

In [18]:
query_gen = load_tweets('stream_ازيك.jsonl')

### Turn query generators into dataframes

In [19]:
def tweet_to_df(tweets):
    data = {'text': [], 'screen_name': [], 'geo': [],
       }
    
    for t in tweets:
    
        data['text'].append(t['text'])
        data['screen_name'].append(t['user']['screen_name'])
        data['geo'].append(t['user']['location'])
    
    return pd.DataFrame(data)

In [20]:
query_df = tweet_to_df(query_gen)

In [21]:
query_df.sample(4)

,geo,screen_name,text
530,مَـ ع ـــآاک دايمــٰــآاا ♥,safa2001gam,بتفتحوا مواضيع تتكلموا فيها ازاي يآا جمااعه ، ...
252,Gharbia.Egypt,MARWA_MUHAMMED1,RT @nourai208: انا الفترة ديه مخنوقة لدرجة لو ...
602,None,NodyOsman,RT @Sam_ar_: انا في موود لو حد قالي ازيك هعيط
304,International,mo7ammed_ali7,أنا الأسبوع دا So depressed لدرجة لو حد قالي ا...


### Clean tweets (removes emojis, links, special characters, and ASCII chars/digits from tweet text)

Using regular expressions such as `[0-9A-Za-z:!/.?]` can filter out much of the unwanted data and retain Arabic characters but still seems to also retain a more recently released generation of emojis. Try updating tweet-preprocessor to version 0.5.0 or using another tweet processing package. 

Different scenarios will require different regex combinations. For example, the below approach wouldn't be of much use if you're building English dialect classifiers.

In [22]:
def cleaner(message):
    message = re.sub("([0-9A-Za-z:!/.])", " ", message)
    return message

In [25]:
cleaned_text = []

for tweet in query_df['text']:
    cleaned_text.append(p.clean(cleaner(tweet)))

query_df['cleaned_text']=cleaned_text

In [26]:
cleaned_name = []

for tweet in query_df['screen_name']:
    cleaned_name.append(p.clean(tweet))

query_df['cleaned_name']=cleaned_name

In [27]:
cleaned_geo = []

for tweet in query_df.geo:
    cleaned_geo.append(p.clean(str(tweet)))

query_df['cleaned_geo']=cleaned_geo


In [28]:

query_df.sample(5)

,geo,screen_name,text,cleaned_text,cleaned_name,cleaned_geo
582,"Cairo, Egypt",Gasser_Mo7amed,RT @Sam_ar_: انا في موود لو حد قالي ازيك هعيط,_ _ انا في موود لو حد قالي ازيك هعيط,Gasser_Mo7amed,"Cairo, Egypt"
163,Tanta,3ra2eeboo,@10_mennat اسخم واوسخ 😂😂\nربنا معاكم يابنتى من...,_ اسخم واوسخ ربنا معاكم يابنتى من كتر المذاكرة...,ra2eeboo,Tanta
309,",تحذفني_ احذفك#انقذ_حياه❤",E__top1,RT @azhary_50: = ازيك ياسطا عامل ايه \n - لا ا...,_ = ازيك ياسطا عامل ايه - لا انجز خش ع المصلحه...,E__top1,",تحذفني_ احذفك"
408,اى مكان فى موسيقى,loletaaa1234,RT @Sam_ar_: انا في موود لو حد قالي ازيك هعيط,_ _ انا في موود لو حد قالي ازيك هعيط,loletaaa1234,اى مكان فى موسيقى
141,القلعة,CaptinNamq,ازيك يا ابو صلاح ، الجمهور السعودي بيحبك وبنشج...,ازيك يا ابو صلاح ، الجمهور السعودي بيحبك وبنشج...,CaptinNamq,القلعة


### Remove duplicated tweets

In [29]:
query_df['cleaned_text'].duplicated().sample(5)

182     True
273    False
11     False
312    False
492    False
Name: cleaned_text, dtype: bool

In [30]:
query_cleaned = query_df.drop_duplicates(['cleaned_text'], keep='last')
query_cleaned = query_cleaned.drop(['screen_name', 'text', 'geo'], axis=1)

In [32]:
query_cleaned.sample(4)

,cleaned_text,cleaned_name,cleaned_geo
429,ازيك مصطفي صباحك فل️,moustafakamelm,None
290,_ اكييد واللي ضاحك مين ازيك ياقلبي,Nnjoood2,None
71,_ هاي ازيك ما تعرفيش اماكن في مصر بتعرض رسومات...,ra2_281,None
314,_ = ازيك ياسطا عامل ايه - لا انجز خش ع المصلحه...,Melesia_achraaf,"Cairo, Egypt"


In [35]:
query_cleaned.fillna('None')

,cleaned_text,cleaned_name,cleaned_geo
0,ازيك صباح الفل,D38647416,None
1,_ _ هلا فيك ازيك,_llin,
3,اللهم آمين آمين آمين ازيك يارب تكون في احسن حال,MohammedTalaat1,"القاهرة, مصر"
4,ازيك يا ادهم سليمان —,mohmedelhnawy24,None
5,محمد علي ازيك,s_ss3,بولندا
6,لما البنت اللي بتحبها تكون واقفه مع ابوها وانت...,adoom101,الاسكندرية
7,ازيك يا محمد قلقتني عليك بشدة اختفائك ؟,Hazem__Azim,Cairo
8,__ ازيك يا محمد قلقتني عليك بشدة اختفائك ؟,mzeezm131,None
9,ازيك يا احلي وزه اعمل حسابك تيجيلي كل يوم الاو...,ZZeko1997,None
10,_ ازيك يامى,PwxRJC67w6wcykP,None


In [36]:
query_cleaned.info() 

<class 'pandas.core.frame.DataFrame'>
Int64Index: 330 entries, 0 to 632
Data columns (total 3 columns):
cleaned_text    330 non-null object
cleaned_name    330 non-null object
cleaned_geo     330 non-null object
dtypes: object(3)
memory usage: 10.3+ KB


### Consolidate different geo tags

In [37]:
locations = {'Saudi Arabia': ['Kingdom of Saudi Arabia', 'MAKKAH', 'saudi arabia', 'jeddah', 'مكة المكرمة', 'المملكة العربية', 'Jeddah', 'Riyadh', 'المملكة العربية السعودية', 'الرياض, المملكة العربية السعودية', 'الرياض, المملكة العربية السعودية', 'jeddah , saudi arabia', 'الرياض', 'آلقصيم ، بريده', 'جدة, المملكة العربية السعودية', 'المدينة المنورة, المملكة العربية السعودية', 'Riyadh, Kingdom of Saudi Arabia', 'makkah - saudi arabia', 'جده', 'Dammam', 'Dammam, Eastern', 'Al Khobar, Kingdom of Saudi Arabia'],
             'Kuwait': ['الكويت', 'Alkuwait', 'kuwait', 'KUWAIT', 'Kuwait , Salwa', 'Qortuba, Kuwait', 'دولة الكويت', 'العاصمه, دولة الكويت', 'The Capital, Kuwait', 'Al Salam, Kuwait', 'Salwa, Kuwait', 'كوكب الشرق - Kuwait'],
             'Egypt': ['Domiat, Egypt', 'Damitta', 'Alexandria, Egypt', 'Damitta-Egypt', 'Giza, Egypt', 'New damieta city', 'مصر', 'القاهرة, مصر', 'El Behera, Egypt', 'الجيزة, مصر', 'Elmansoura', 'El Menia, Egypt', 'El Qaliobia, Egypt', 'البحيرة, مصر', 'Domyat elgadeda', 'El Sharkia, Egypt', 'sharkia_fakous', 'Cairo, Egypt', 'Alexandria,Egypt', 'ELsharkia , Egypt', 'شمال سيناء, مصر', 'كفر الشيخ, مصر', 'Port Said, Egypt', 'Beni Suef, Egypt', 'القليوبية, مصر', 'بنها']
            }

In [38]:
'MAKKAH' in list(locations.values())[0]

True

In [39]:
'MAKKAH' in list(locations.values())[1]

False

In [40]:
locations.values()

dict_values([['Kingdom of Saudi Arabia', 'MAKKAH', 'saudi arabia', 'jeddah', 'مكة المكرمة', 'المملكة العربية', 'Jeddah', 'Riyadh', 'المملكة العربية السعودية', 'الرياض, المملكة العربية السعودية', 'الرياض, المملكة العربية السعودية', 'jeddah , saudi arabia', 'الرياض', 'آلقصيم ، بريده', 'جدة, المملكة العربية السعودية', 'المدينة المنورة, المملكة العربية السعودية', 'Riyadh, Kingdom of Saudi Arabia', 'makkah - saudi arabia', 'جده', 'Dammam', 'Dammam, Eastern', 'Al Khobar, Kingdom of Saudi Arabia'], ['الكويت', 'Alkuwait', 'kuwait', 'KUWAIT', 'Kuwait , Salwa', 'Qortuba, Kuwait', 'دولة الكويت', 'العاصمه, دولة الكويت', 'The Capital, Kuwait', 'Al Salam, Kuwait', 'Salwa, Kuwait', 'كوكب الشرق - Kuwait'], ['Domiat, Egypt', 'Damitta', 'Alexandria, Egypt', 'Damitta-Egypt', 'Giza, Egypt', 'New damieta city', 'مصر', 'القاهرة, مصر', 'El Behera, Egypt', 'الجيزة, مصر', 'Elmansoura', 'El Menia, Egypt', 'El Qaliobia, Egypt', 'البحيرة, مصر', 'Domyat elgadeda', 'El Sharkia, Egypt', 'sharkia_fakous', 'Cairo, Egy

In [41]:
for word in query_cleaned['cleaned_geo'].str.split('_'):
    if word[0] in list(locations.values())[0]:
        query_cleaned['cleaned_geo'].replace(word, 'Saudi Arabia', inplace=True)
    elif word[0] in list(locations.values())[1]:
            query_cleaned['cleaned_geo'].replace(word, 'Kuwait', inplace=True)
    else:
        if word[0] in list(locations.values())[2]:
            query_cleaned['cleaned_geo'].replace(word, 'Egypt', inplace=True)
            

In [43]:
query_cleaned.sample(5)

,cleaned_text,cleaned_name,cleaned_geo
429,: ازيك مصطفي صباحك فل️,moustafakamelm,None
449,: مرة شوية رز حبو يسلمو على مديرهم قالوله ازيك...,rahafbadarneh,المملكة الأردنية الهاشمية
367,: ازيك و أزي المدام و الولاد,sanaasan333,None
472,حتى ازيك مستخسرها مستخسرها مبيقولهاش,KhaledG06308330,mansoura
125,ازيك يا اتش كيفك,wLsRdyB6q14ke8x,None


### Pickle to conduct EDA in seperate notebook

In [1]:
pwd

'/home/jovyan/capstone-34/34.210.104.115/misc'

In [44]:
query_cleaned.to_pickle('../eg_twitter_pickled/stream_ازيك.p')